In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
!pip install random-user-agent

     |████████████████████████████████| 8.2MB 8.1MB/s 


In [6]:
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from random_user_agent.user_agent import UserAgent
from random_user_agent.params import SoftwareName, OperatingSystem
from selenium import webdriver

In [36]:
def get_user_agent():
  software_names = [SoftwareName.CHROME.value]
  operating_systems = [OperatingSystem.WINDOWS.value, OperatingSystem.LINUX.value]   
  user_agent_rotator = UserAgent(software_names=software_names, operating_systems=operating_systems, limit=100)

      # Get list of user agents.
  user_agents = user_agent_rotator.get_user_agents()

      # Get Random User Agent String.
  user_agent = user_agent_rotator.get_random_user_agent()
  return user_agent

In [42]:
get_user_agent()

'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.90 Safari/537.36 2345Explorer/9.4.3.17934'

In [52]:
def create_driver():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  chrome_options.add_argument(f'user-agent={get_user_agent()}')
  chrome_options.add_argument('--window-size=1420,1080')
  driver = webdriver.Chrome('chromedriver',options=chrome_options)
  return driver

In [54]:
driver = create_driver()
type(driver)

selenium.webdriver.chrome.webdriver.WebDriver

In [56]:
driver.get('https://translate.google.com/#view=home&op=translate&sl=en&tl=fr')
#driver.find_element_by_xpath('/html/body/div/c-wiz/div/div/div/div[2]/div[1]/div[4]/form/div[1]/div/button').click()

In [57]:
driver.get("https://translate.google.com/?sl=en&tl=pt&op=translate")
words = ['sleep']
text_area = driver.find_elements_by_tag_name("textarea")[0]
text_area.clear()
text_area.send_keys(words[0])
time.sleep(1)
trans_word_ele = WebDriverWait(driver=driver, timeout=1).until(lambda d : d.find_element_by_xpath('/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[2]/div[5]/div/div[1]/span[1]/span/span'))
print(trans_word_ele.text)
driver.find_element(By.TAG_NAME, 'table').text

dormir


'Part of speech\nTranslation\nReverse translations\nFrequency\nhelp_outline\nverb\ndormir\nsleep, fall asleep, rest, lie, kip, doss\nadormecer\nfall asleep, sleep, go numb, drop out, doss\ndescansar\nrest, sleep, take a rest, lounge, breathe, recess\nrepousar\nrest, lie, sleep, lean, recline, doze\nestar a dormir\nsleep\nestar inativo\ndoss, sleep\nestar sepultado\nsleep\nnoun\no sono\nsleep, rest, sleepiness, repose, sandman, time-out\no descanso\nrest, pillow, break, relaxation, sleep, respite\no período de sono\nsleep\na paz\npeace, repose, sleep, halcyon\na tranqüilidade\ntranquility, quiet, calm, peace, ease, sleep'

In [58]:
def raw_text_to_text(tabel, number_pos):
    dict_trans = {}
    for index in range(number_pos):
        all_trans = tabel.find_elements(By.TAG_NAME, 'tbody')[index].text.split('\n')
        pos = all_trans[0]
        number_word = len(all_trans)
        words = [all_trans[i] for i in range(number_word) if i%2 == 1]
        dict_trans[pos] = words
    return dict_trans

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/data/Lang_Word/EN_vocab/all.csv")
words = list(df['source'].values)
words = [str(x) for x in words]
words = list(set(words))[:20000]
my_dict = []
driver = create_driver()
driver.get("https://translate.google.com/?sl=en&tl=pt&op=translate")

f = open('/content/drive/MyDrive/data/en-pt.txt', 'a')
text_area = driver.find_elements_by_tag_name("textarea")[0]
for word in words:
    print(word, end=' ->  ')
    text_area.clear()
    text_area.send_keys(word)
    time.sleep(1)
    try:
      trans_word_ele = WebDriverWait(driver=driver, timeout=1).until(lambda d : d.find_element_by_xpath('/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[2]/div[5]/div/div[1]/span[1]/span/span'))
    except:
      try:
        trans_word_ele = driver.find_element_by_xpath('/html/body/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div[1]/div[2]/div[2]/c-wiz[2]/div[5]/div[2]/div[1]/span[1]')
      except:
        driver = create_driver()
        driver.get('https://www.google.com/')
        time.sleep(5)
        driver.get("https://translate.google.com/?sl=en&tl=fr&op=translate")
        text_area = driver.find_elements_by_tag_name("textarea")[0]
        continue
    trans_word = trans_word_ele.text
    print(trans_word)
    try:
      table = driver.find_element(By.TAG_NAME, 'table')
      number_pos = len(table.find_elements(By.TAG_NAME, 'tbody'))
      temp_dict = raw_text_to_text(table, number_pos)
      for pos in list(temp_dict.keys()):
        words_trans = temp_dict[pos]
        for w_trans in words_trans:
          line_dict = word + '\t'+ w_trans + '\t'+pos
          my_dict.append(line_dict)
          f.write("%s\n" % line_dict)
      my_dict.append(word + '\t'+ trans_word + '\t'+'None')
      f.write("%s\n" % (word + '\t'+ trans_word + '\t'+'None'))
    except:
        my_dict.append(word + '\t'+ trans_word + '\t'+'properNoun')
        f.write("%s\n" % (word + '\t'+ trans_word + '\t'+'properNoun'))
f.close()

Einir ->  Einir.
Loayza ->  Loayza.
Primitivo ->  Primitivo
hemoblast ->  hemoblast
Auel ->  Auel.
firewoman ->  bombeira
sepiolite ->  SEPIOLITE.
exasperation ->  exasperação
Tarbagatay ->  Tarbagatay.
Cape-Verde ->  Cabo Verde
Emporiki ->  Emporiki.
Gorbis ->  Gorbis.
Marró ->  Marrógrafo
Medel ->  Medel.
Shogun ->  Shogun
Enes ->  Enes.
transoceanic ->  transoceânico
Asrim ->  Asrim.
Veliko Tarnovo ->  Veliko tarnovo.
industrial state ->  estado industrial
Teodora ->  Teodora.
shinpad ->  shinpad.
smoked ->  defumado
Paride ->  Paride
Ertugan ->  Ertugan.
recount ->  recontagem
Pechora ->  Pecora.
Anhalt ->  Anhalt
except for ->  exceto por
Sib ->  Sib.
away from ->  longe de
amortizable ->  Amortizável
Richards ->  Richards.
landing strip ->  Landing Strip
Elgin ->  Elgin.
Ruecker ->  Ruecker.
Biridina ->  Biridina
Gezim ->  Gezim.
dapple ->  deple.
Zeyid ->  Zeyid.
well-deserved ->  bem merecido
shinguard ->  caneleira
pizza ->  pizza
Miklavcic ->  Miklavcic.
reappoint ->  reavali